# Usage and Functionality
This notebook generates synthetic micromodel images composed of bead-like structures.
Run the cells sequentially to create TIFF images and HDF5 datasets in the `image_output/` directory under the current working directory.
You can adjust parameters such as `rad`, `stride`, and deviation values in each code cell to create different geometries.
Later sections demonstrate more complex models including high-permeability regions and overlays of large beads.
Ensure required Python packages (`matplotlib`, `numpy`, `h5py`, `skimage`, and `Pillow`) are installed before execution.


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patch
import random
import numpy as np
from matplotlib.backends.backend_agg import FigureCanvasAgg
from skimage import data
import h5py
import os


In [ ]:
# Draws circles on a offset grid with a radius(rad), deviation in radius, and deviation in center of the circle
# Deviations are random integers(pixels) between zero and devmax.
# Saves the model to hdf5 along with the location and radius of each circle
wd = os.getcwd()
output_direc = "/image_output/"

rad = 12
stride = 40
offset = 20
x_dim = 1200
y_dim = 1200
xdevmax = 12
ydevmax = 12
raddevmax = 6
target_direc = wd+output_direc

for r in range(0,13,1):
    #raddevmax = r
    for c in range(0,1,1):
        #xdevmax = c
        #ydevmax = c
        x_coor = []
        y_coor = []
        r_coor = []
        for j in range(0,x_dim+offset,stride):
            counter = 0
            for k in range(0, y_dim+offset, stride):
                if (counter % 2) == 0:
                    x_coor.append(j+random.randint(-xdevmax,xdevmax))
                    y_coor.append(k+random.randint(-ydevmax,ydevmax))
                    r_coor.append(rad+random.randint(-raddevmax,raddevmax))
                    x_coor.append(j+random.randint(-xdevmax,xdevmax))
                    y_coor.append(k+stride+random.randint(-ydevmax,ydevmax))
                    r_coor.append(rad+random.randint(-raddevmax,raddevmax))
                else:
                    x_coor.append(j+offset+random.randint(-xdevmax,xdevmax))
                    y_coor.append(k+offset+random.randint(-ydevmax,ydevmax))
                    r_coor.append(rad+random.randint(-raddevmax,raddevmax))
                    x_coor.append(j-offset+random.randint(-xdevmax,xdevmax))
                    y_coor.append(k-offset+random.randint(-ydevmax,ydevmax))
                    r_coor.append(rad+random.randint(-raddevmax,raddevmax))
                counter = counter+1

        fig, ax = plt.subplots()
        ax.set_xlim((0, x_dim))
        ax.set_ylim((0, y_dim))
        ax.set_aspect('equal')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        ax.axis('off')
        DPI = fig.get_dpi()
        fig.set_size_inches(x_dim/float(DPI),y_dim/float(DPI))

        for xc, yc, rc in zip(x_coor, y_coor, r_coor):
            circle = patch.Circle((xc, yc), rc, color='black', fill=True, linewidth = 0)
            ax.add_artist(circle)

        
        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        width, height = canvas.get_width_height()
        img = np.frombuffer(canvas.tostring_rgb(), dtype=np.uint8)
        img = img.reshape((height, width, 3))
        binary = (img > 254)*1
        binary2 = binary[:, :, 0]
        Porosity = (binary2 == 1).sum()/((binary2 == 0).sum()+(binary2 == 1).sum())

        with h5py.File(target_direc+"/"+"perfectlyoffsetcircles_rad"+str(rad*10)+"_raddev"+str(raddevmax*10)+"_coordev"+str(xdevmax*10)+"_repeat"+str(r)+"_BIG.hdf5", 'w') as g:
            g.create_dataset('x_coor', data=np.array(x_coor)*10, dtype='float', compression='gzip')
            g.create_dataset('y_coor', data=np.array(x_coor)*10, dtype='float', compression='gzip')
            g.create_dataset('rad', data=np.array(r_coor)*10, dtype='uint16', compression='gzip')
            g.create_dataset('binary_image', data=binary2, dtype='uint8', compression='gzip')
            g.attrs['porosity'] = Porosity
            g.attrs['rad'] = rad*10
            g.attrs['stride'] = stride*10
            g.attrs['offset'] = offset*10
            g.attrs['xdevmax'] = xdevmax*10
            g.attrs['ydevmax'] = ydevmax*10
            g.attrs['raddevmax'] = raddevmax*10





In [ ]:
# Create a micromodel with a region of high perm sandwiched in between two regions of low perm

rad1 = 7
rad2 = 9
stride = 20
offset = 10
x_coor = []
y_coor = []
x_dim = 510
y_dim = 510
for j in range(0,x_dim,stride):
    counter = 0
    for k in range(0, y_dim, stride):
        if (counter % 2) == 0:
            x_coor.append(k)
            y_coor.append(j)
        else:
            x_coor.append(k)
            y_coor.append(j+offset)
        counter = counter+1

fig, ax = plt.subplots()
ax.set_xlim((0, x_dim))
ax.set_ylim((0, y_dim))
ax.set_aspect('equal')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax.axis('off')
DPI = fig.get_dpi()
fig.set_size_inches(x_dim/float(DPI),y_dim/float(DPI))


for idx, (xc, yc) in enumerate(zip(x_coor, y_coor)):
    offset_x = random.randint(0,12)
    offset_y = random.randint(0,12)
    if idx < len(x_coor)/3:
        radius = rad2
    elif idx < len(x_coor)/3*2:
        radius = rad1
    else:
        radius = rad2
    circle = plt.Circle((xc+offset_x, yc+offset_y), radius, color='black')
    ax.add_artist(circle)




In [ ]:
# Creates microporous Oolites with solid interiors

rad1 = 7
rad1maxdev = 1
rad2 = 10
rad2maxdev = 2
stride = 20
offset = 10
x_coor = []
xmaxdev = 12
xmaxmicrodev = 2
y_coor = []
ymaxdev = 12
ymaxmicrodev = 2
x_dim = 1020
y_dim = 1020
for j in range(0,x_dim,stride):
    counter = 0
    for k in range(0, y_dim, stride):
        if (counter % 2) == 0:
            x_coor.append(k+random.randint(0,xmaxdev))
            y_coor.append(j+random.randint(0,ymaxdev))
        else:
            x_coor.append(k+random.randint(0,xmaxdev))
            y_coor.append(j+offset+random.randint(0,ymaxdev))
        counter = counter+1

fig, ax = plt.subplots()
ax.set_xlim((0, x_dim))
ax.set_ylim((0, y_dim))
ax.set_aspect('equal')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax.axis('off')
DPI = fig.get_dpi()
fig.set_size_inches(x_dim/float(DPI),y_dim/float(DPI))
for xc, yc, rc in zip(x_coor, y_coor, r_coor):
            circle = patch.Circle((xc, yc), rc, color='black', fill=True, linewidth = 0)
            ax.add_artist(circle)
for xc, yc, rc in zip(x_coor, y_coor, r_coor):
            circle = patch.Circle((xc, yc), rc, color='black', fill=True, linewidth = 0)
            ax.add_artist(circle)


In [ ]:
# Draw gigantic micromodel
# Draws circles on a offset grid with a radius(rad), deviation in radius, and deviation in center of the circle
# Deviations are random integers(pixels) between zero and devmax.
# Saves the model to hdf5 along with the location and radius of each circle
wd = os.getcwd()
output_direc = "/image_output/"

rad = 6
stride = 20
offset = 10
x_dim = 2400
y_dim = 2400
xdevmax = 6
ydevmax = 6
raddevmax = 3
target_direc = wd+output_direc

for r in range(0,13,1):
    #raddevmax = r
    for c in range(0,1,1):
        #xdevmax = c
        #ydevmax = c
        x_coor = []
        y_coor = []
        r_coor = []
        for j in range(0,x_dim+offset,stride):
            counter = 0
            for k in range(0, y_dim+offset, stride):
                if (counter % 2) == 0:
                    x_coor.append(j+random.randint(-xdevmax,xdevmax))
                    y_coor.append(k+random.randint(-ydevmax,ydevmax))
                    r_coor.append(rad+random.randint(-raddevmax,raddevmax))
                    x_coor.append(j+random.randint(-xdevmax,xdevmax))
                    y_coor.append(k+stride+random.randint(-ydevmax,ydevmax))
                    r_coor.append(rad+random.randint(-raddevmax,raddevmax))
                else:
                    x_coor.append(j+offset+random.randint(-xdevmax,xdevmax))
                    y_coor.append(k+offset+random.randint(-ydevmax,ydevmax))
                    r_coor.append(rad+random.randint(-raddevmax,raddevmax))
                    x_coor.append(j-offset+random.randint(-xdevmax,xdevmax))
                    y_coor.append(k-offset+random.randint(-ydevmax,ydevmax))
                    r_coor.append(rad+random.randint(-raddevmax,raddevmax))
                counter = counter+1

        fig, ax = plt.subplots()
        ax.set_xlim((0, x_dim))
        ax.set_ylim((0, y_dim))
        ax.set_aspect('equal')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        ax.axis('off')
        DPI = fig.get_dpi()
        fig.set_size_inches(x_dim/float(DPI),y_dim/float(DPI))

        for xc, yc, rc in zip(x_coor, y_coor, r_coor):
            circle = patch.Circle((xc, yc), rc, color='black', fill=True, linewidth = 0)
            ax.add_artist(circle)

        
        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        width, height = canvas.get_width_height()
        img = np.frombuffer(canvas.tostring_rgb(), dtype=np.uint8)
        img = img.reshape((height, width, 3))
        binary = (img > 254)*1
        binary2 = binary[:, :, 0]
        Porosity = (binary2 == 1).sum()/((binary2 == 0).sum()+(binary2 == 1).sum())

        with h5py.File(target_direc+"/"+"perfectlyoffsetcircles_rad"+str(rad*10)+"_raddev"+str(raddevmax*10)+"_coordev"+str(xdevmax*10)+"_repeat"+str(r)+"_BIG.hdf5", 'w') as g:
            g.create_dataset('x_coor', data=np.array(x_coor)*10, dtype='float', compression='gzip')
            g.create_dataset('y_coor', data=np.array(x_coor)*10, dtype='float', compression='gzip')
            g.create_dataset('rad', data=np.array(r_coor)*10, dtype='uint16', compression='gzip')
            g.create_dataset('binary_image', data=binary2, dtype='uint8', compression='gzip')
            g.attrs['porosity'] = Porosity
            g.attrs['rad'] = rad*10
            g.attrs['stride'] = stride*10
            g.attrs['offset'] = offset*10
            g.attrs['xdevmax'] = xdevmax*10
            g.attrs['ydevmax'] = ydevmax*10
            g.attrs['raddevmax'] = raddevmax*10




In [ ]:
# add overlay of XXL beads as a mask to gigantic images
wd = os.getcwd()
output_direc = "/image_output/"
rad = 6
stride = 20
offset = 10
x_dim = 2400
y_dim = 2400
xdevmax = 6
ydevmax = 6
raddevmax = 3
target_direc = wd+output_direc

from PIL import Image
Image.MAX_IMAGE_PIXELS = None
r=1


rad2 = 60
stride2 = 200
offset2 = 100
x_dim2 = 2400
y_dim2 = 2400
xdevmax2 = 60
ydevmax2 = 60
raddevmax2 = 30

x_coor = []
y_coor = []
r_coor = []
for j in range(0,x_dim2+offset2,stride2):
    counter = 0
    for k in range(0, y_dim2+offset2, stride2):
        if (counter % 2) == 0:
            x_coor.append(j+random.randint(-xdevmax2,xdevmax2))
            y_coor.append(k+random.randint(-ydevmax2,ydevmax2))
            r_coor.append(rad2+random.randint(-raddevmax2,raddevmax2))
            x_coor.append(j+random.randint(-xdevmax2,xdevmax2))
            y_coor.append(k+stride+random.randint(-ydevmax2,ydevmax2))
            r_coor.append(rad2+random.randint(-raddevmax2,raddevmax2))
        else:
            x_coor.append(j+offset2+random.randint(-xdevmax2,xdevmax2))
            y_coor.append(k+offset2+random.randint(-ydevmax2,ydevmax2))
            r_coor.append(rad2+random.randint(-raddevmax2,raddevmax2))
            x_coor.append(j-offset2+random.randint(-xdevmax2,xdevmax2))
            y_coor.append(k-offset2+random.randint(-ydevmax2,ydevmax2))
            r_coor.append(rad2+random.randint(-raddevmax2,raddevmax2))
        counter = counter+1

fig, ax = plt.subplots()
ax.set_xlim((0, x_dim2))
ax.set_ylim((0, y_dim2))
ax.set_aspect('equal')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax.axis('off')
DPI = fig.get_dpi()
fig.set_size_inches(x_dim/float(DPI),y_dim/float(DPI))

for xc, yc, rc in zip(x_coor, y_coor, r_coor):
            circle = patch.Circle((xc, yc), rc, color='black', fill=True, linewidth = 0)
            ax.add_artist(circle)


        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        width, height = canvas.get_width_height()
        img = np.frombuffer(canvas.tostring_rgb(), dtype=np.uint8)
        img = img.reshape((height, width, 3))
        binary = (img > 254)*1
        binary2 = binary[:, :, 0]
        Porosity = (binary2 == 1).sum()/((binary2 == 0).sum()+(binary2 == 1).sum())

        with h5py.File(target_direc+"/"+"perfectlyoffsetcircles_rad"+str(rad*10)+"_raddev"+str(raddevmax*10)+"_coordev"+str(xdevmax*10)+"_repeat"+str(r)+"_BIG.hdf5", 'w') as g:
            g.create_dataset('x_coor', data=np.array(x_coor)*10, dtype='float', compression='gzip')
            g.create_dataset('y_coor', data=np.array(x_coor)*10, dtype='float', compression='gzip')
            g.create_dataset('rad', data=np.array(r_coor)*10, dtype='uint16', compression='gzip')
            g.create_dataset('binary_image', data=binary2, dtype='uint8', compression='gzip')
            g.attrs['porosity'] = Porosity
            g.attrs['rad'] = rad*10
            g.attrs['stride'] = stride*10
            g.attrs['offset'] = offset*10
            g.attrs['xdevmax'] = xdevmax*10
            g.attrs['ydevmax'] = ydevmax*10
            g.attrs['raddevmax'] = raddevmax*10

plt.imshow(micro_beads)



In [ ]:
plt.imshow(micro_beads)

In [ ]:
# file already closed via context manager
